author: leezeeyee   
date: 2021/4/13  
link: [github](https://github.com/easilylazy/pattern-recognition) 

In [2]:
import pandas as pd
import numpy as np

In [3]:
irisData=pd.read_csv('iris.csv')
irisData.head()

,Unnamed: 0,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,1,5.1,3.5,1.4,0.2,setosa
1,2,4.9,3.0,1.4,0.2,setosa
2,3,4.7,3.2,1.3,0.2,setosa
3,4,4.6,3.1,1.5,0.2,setosa
4,5,5.0,3.6,1.4,0.2,setosa


In [4]:
## different types
setosas=irisData[irisData['Species']=='setosa']
versicolors=irisData[irisData['Species']=='versicolor']
virginicas=irisData[irisData['Species']=='virginica']

In [35]:
classes=3
Y=np.r_[np.zeros(len(setosas),dtype=int),np.ones(len(versicolors), dtype=int),2*np.ones(len(virginicas), dtype=int)]
one_hot_Y = np.eye(classes)[Y]


In [53]:
X=np.asarray(irisData[['Sepal.Length',	'Sepal.Width',	'Petal.Length',	'Petal.Width']].iloc[:,:])

In [70]:
trainX=[]
trainY=[]
testX=[]
testY=[]

for i in range(classes):
    index=np.where(Y==i)[0]
    np.random.shuffle(index)
    
    trainX.append(X[index[:30]])
    testX.append(X[index[30:]])
    trainY.append(one_hot_Y[index[:30]])
    testY.append(one_hot_Y[index[30:]])
    print(index)
print(trainX[2][0])
print(X[index[0]])
trainX_sh=np.r_[trainX[0],trainX[1],trainX[2]]
trainY_sh=np.r_[trainY[0],trainY[1],trainY[2]]
testX_sh=np.r_[testX[0],testX[1],testX[2]]
testY_sh=np.r_[testY[0],testY[1],testY[2]]
trainX_sh[:3]

[ 1 16 14  5  9 40  8 48 27  0 28 13 47 19 12 15 11 30  3 49 35 45 10 17
  7 34 43 46 24 33 26 37 23  4 21 42 44 18 29 32 39 41 31 20 22 38  6 36
 25  2]
[51 82 77 86 89 76 94 93 72 57 71 78 64 75 55 59 68 81 97 83 73 95 69 50
 91 58 90 96 88 60 85 67 80 62 54 92 98 56 65 52 66 84 61 70 53 74 99 87
 79 63]
[145 110 128 144 129 131 106 138 135 140 130 115 149 142 113 124 139 118
 105 120 133 122 117 132 134 143 101 136 109 114 119 125 137 108 100 126
 127 104 147 123 116 121 107 148 112 111 141 103 102 146]
[6.7 3.  5.2 2.3]
[6.7 3.  5.2 2.3]


array([[4.9, 3. , 1.4, 0.2],
       [5.4, 3.9, 1.3, 0.4],
       [5.8, 4. , 1.2, 0.2]])

In [74]:
def softmax(X,Y,K,N,W,M):
    ## compute si
    softMaxY=np.zeros(K)
    # step size
    stepSize=1
    # initialize W
    W=np.zeros([K,N])
    for step in range(30):
        for i in range(M):
            # print(np.dot(W,np.transpose(X[i])))
            S=np.dot(W,np.transpose(X[i]))
            # print(np.dot(S,np.transpose(Y[i])))
            
            # print(np.where(y==1)[0][0]==np.where(S==np.max(S))[0][0])
            softMaxS=np.exp(S)
            sumY=np.sum(softMaxS)
            # print(sumY)
            # print(softMaxY)
            for j in range(K):
                softMaxY[j]=softMaxS[j]/sumY
                if Y[i][j]==1:
                    W[j]-=stepSize*(softMaxY[j]-1)*X[i]
                else:
                    W[j]-=stepSize*(softMaxY[j])*X[i]
            # print(step)
            # print(softMaxY)
            # print(W)

        Pass=True
        for i in range(M):
            y=Y[i]
            S=np.dot(W,np.transpose(X[i]))
            if np.where(y==1)[0][0] != np.where(S==np.max(S))[0][0]:
                Pass=False
        if Pass is True:
            print('success')
            break
    return W

In [75]:
softmax(X=trainX_sh,Y=trainY_sh,K=K,N=N,W=W,M=M)

success


array([[ 3.26666667,  2.        ,  0.93333333,  0.13333333],
       [-1.63333333, -1.        , -0.46666667, -0.06666667],
       [-1.63333333, -1.        , -0.46666667, -0.06666667]])

In [33]:
type(Y[0])

numpy.int32

In [7]:
targets 

array([2, 3, 4, 0])

In [23]:
type(targets[0])

numpy.int32

In [59]:
# k for class num; N for dimension of data
K=3
N=4
W=np.zeros([K,N])
# M for the item num of data
M=4

## data